## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd

charity1_df = pd.read_csv("Resources/charity_data.csv")
charity1_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

charity2_df = charity1_df.drop(['EIN', 'NAME'], axis=1, inplace=False)
charity2_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.

for col in charity2_df.columns:
    print(col, len(charity2_df[col].unique().tolist()))

APPLICATION_TYPE 17
AFFILIATION 6
CLASSIFICATION 71
USE_CASE 5
ORGANIZATION 4
STATUS 2
INCOME_AMT 9
SPECIAL_CONSIDERATIONS 2
ASK_AMT 8747
IS_SUCCESSFUL 2


In [4]:
# Look at APPLICATION_TYPE value counts for binning

application_types = charity2_df['APPLICATION_TYPE'].value_counts()
application_types

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

application_types_to_replace = list(application_types[application_types < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    charity2_df['APPLICATION_TYPE'] = charity2_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
charity2_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning

class_types = charity2_df['CLASSIFICATION'].value_counts()
class_types

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C1236        1
C6100        1
C4500        1
C8210        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1

class_types[class_types >= 20]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

class_types_to_replace = list(class_types[class_types < 300].index)

# Replace in dataframe
for cls in class_types_to_replace:
    charity2_df['CLASSIFICATION'] = charity2_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
charity2_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`

charity3_df = pd.get_dummies(charity2_df)
charity3_df.shape

(34299, 45)

In [18]:
# Split our preprocessed data into our features and target arrays

y = charity3_df['IS_SUCCESSFUL'].values
X = charity3_df.drop(['IS_SUCCESSFUL'], 1).values

# Split the preprocessed data into a training and testing dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

len(X_train[0])

44

## Compile, Train and Evaluate the Model

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train[0])
hidden_nodes_layer1 =  22
hidden_nodes_layer2 = 11

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 22)                990       
                                                                 
 dense_1 (Dense)             (None, 11)                253       
                                                                 
 dense_2 (Dense)             (None, 1)                 12        
                                                                 
Total params: 1,255
Trainable params: 1,255
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [24]:
# Train the model

fit_model = nn.fit(X_train_scaled,y_train,epochs=100, verbose=0)

In [21]:
# Evaluate the model using the test data

model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5573 - accuracy: 0.7276 - 110ms/epoch - 411us/step
Loss: 0.5572561025619507, Accuracy: 0.727580189704895


In [26]:
# Evaluate differect node sizes

number_input_features = len(X_train[0])

node_counts = [(160, 80), (120, 60), (100, 50), (80, 40), (60, 30), (40, 20), (20, 10)]

for i in node_counts:
    
    # Define the model
    hidden_nodes_layer1 =  i[0]
    hidden_nodes_layer2 = i[1]

    nn = tf.keras.models.Sequential()

    # First hidden layer
    nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

    # Second hidden layer
    nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

    # Output layer
    nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

    # Train the model
    fit_model = nn.fit(X_train_scaled,y_train,epochs=100, verbose=0)

    # Evaluate the model using the test data
    model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Nodes: {i}, Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5692 - accuracy: 0.7308 - 161ms/epoch - 602us/step
Nodes: (160, 80), Loss: 0.5692409873008728, Accuracy: 0.7308454513549805
268/268 - 0s - loss: 0.5616 - accuracy: 0.7299 - 206ms/epoch - 769us/step
Nodes: (120, 60), Loss: 0.5615937113761902, Accuracy: 0.729912519454956
268/268 - 0s - loss: 0.5588 - accuracy: 0.7297 - 195ms/epoch - 727us/step
Nodes: (100, 50), Loss: 0.5588234066963196, Accuracy: 0.72967928647995
268/268 - 0s - loss: 0.5628 - accuracy: 0.7297 - 186ms/epoch - 695us/step
Nodes: (80, 40), Loss: 0.5627895593643188, Accuracy: 0.72967928647995
268/268 - 0s - loss: 0.5558 - accuracy: 0.7315 - 182ms/epoch - 679us/step
Nodes: (60, 30), Loss: 0.5558165907859802, Accuracy: 0.7315452098846436
268/268 - 0s - loss: 0.5566 - accuracy: 0.7303 - 184ms/epoch - 685us/step
Nodes: (40, 20), Loss: 0.5566084980964661, Accuracy: 0.7302623987197876
268/268 - 0s - loss: 0.5526 - accuracy: 0.7286 - 187ms/epoch - 700us/step
Nodes: (20, 10), Loss: 0.5526009202003479, Accuracy:

In [27]:
# Export our model to HDF5 file

nn.save("charity_model.h5")